Materia: Procesamiento de Imagénes y Visión por Computadora


Alumna: Menescaldi, Brisa

###Ejercicio 4
Con los videos de youtube.com de cámara fija pedidos para esta clase, aplicar los algoritmos de detección de movimiento vistos en la teoría.

In [1]:
!pip install pytube
!apt update && apt install -y handbrake
!pip install -q mediapy

import mediapy as media
from pytube import YouTube
import cv2
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 814.9 kB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,357 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64

In [2]:
url = 'https://www.youtube.com/watch?v=lLoXv2g7KOY'

yt = YouTube(url)

#toma el stream con resolución de 720p
video = yt.streams.filter(progressive=True, file_extension='mp4', res="720p").first()

# archivo de salida
output_filename = 'highway.mp4'

video.download(filename=output_filename)
print("Descarga completada.")

#convertir el video a un formato liviano de trabajo
!ffmpeg -y -i highway.mp4 -vf "scale=600:-1" -an -t 30 highway_600.mp4

Descarga completada.
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis -

In [3]:
url = 'highway_600.mp4'
video = media.read_video(url)
media.show_video(video)

This browser does not support the video tag.


In [4]:
#oscurecer una imagen
def process_image(new_image, prev_image, **kwargs):
    # convertir la imagen a float32
    new_image_float = new_image.astype(np.float32)

    # reducir el brillo a la mitad
    new_image_float *= 0.5

    # convertir la imagen de vuelta a uint8
    new_image_uint8 = np.clip(new_image_float, 0, 255).astype(np.uint8)

    return new_image_uint8

def draw_contours(frame, contours, color=(0, 255, 0), thickness=2):
    # si la imagen es en escala de grises (1 canal)
    if len(frame.shape) == 2 or frame.shape[2] == 1:
        # pasar imagen de escala de grises a color (3 canales)
        result_image = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    else:
        # ya es una imagen de color, hacer una copia
        result_image = frame.copy()

    # dibujar cada contorno en la imagen
    for contour in contours:
        #rectángulo delimitador para cada contorno
        x, y, w, h = cv2.boundingRect(contour)
        # dibujar el rectángulo
        cv2.rectangle(result_image, (x, y), (x + w, y + h), color, thickness)

    return result_image

# procesar un video
def video_processor(filename_in, filename_out, process_func, max_time=10, **kwargs):
    # Abrir el video de entrada para lectura
    with media.VideoReader(filename_in) as r:
        # archivo de video de salida
        with media.VideoWriter(filename_out, shape=r.shape, fps=r.fps, bps=r.bps) as w:
            count = 0  # contador de fotogramas
            prev_image = None  # imagen previa

            # iterar sobre cada imagen (fotograma) del video
            for image in r:
                new_image = media.to_uint8(image)  # imagen a formato flotante

                # comprobar si es la primera imagen
                if prev_image is None:
                    prev_image = new_image.copy()

                # procesar la imagen utilizando la función dada
                processed_image = process_func(new_image, prev_image, **kwargs)

                # Añadir la imagen procesada al video de salida
                w.add_image(processed_image)

                # actualizar la imagen previa
                prev_image = new_image.copy()

                # incrementar el contador de fotogramas
                count += 1

                # detener el proceso si se alcanza el tiempo máximo
                if count >= max_time * r.fps:
                    break

# archivos de video de entrada y salida
filename_in = 'highway_600.mp4'
filename_out = 'highway_dark.mp4'

# procesar el video
video_processor(filename_in, filename_out, process_image, 10)

# video resultante
media.show_video(media.read_video(filename_out), fps=30)

This browser does not support the video tag.


In [5]:
# realizar diferencia de fotogramas con normalización
def process_frame_difference(new_image, prev_image, **kwargs):
    # imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 0, 255, cv2.NORM_MINMAX)

    # umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(norm_diff, 30, 255, cv2.THRESH_BINARY)

    # imagen umbralizada a color, para mantener la consistencia con el video original
    thresh_color = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)

    return thresh_color

# archivos de video de entrada y salida
filename_in = 'highway_600.mp4'
filename_out = 'highway_frame_difference.mp4'

# procesar el video
video_processor(filename_in, filename_out, process_frame_difference, 10)

# video resultante
media.show_video(media.read_video(filename_out), fps=30)

This browser does not support the video tag.


In [6]:
# detectar movimientos y dibujar cuadros delimitadores
def process_frame_difference_full(new_image, prev_image, **kwargs):
    # imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 0, 255, cv2.NORM_MINMAX)

    # umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(norm_diff, 30, 255, cv2.THRESH_BINARY)

    # dilatar la imagen umbralizada, para mejorar la detección de contornos
    kernel = np.ones((5,5),np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations = 1)

    # convertir la imagen dilatada a formato adecuado para findContours
    dilated = dilated.astype(np.uint8)

    # encontrar contornos en la imagen dilatada
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # cuadros delimitadores alrededor de los contornos
    if kwargs.get('draw_mode', 0) == 0:
      result_image = draw_contours(new_image, contours)
    elif kwargs.get('draw_mode', 0) == 1:
      result_image = draw_contours(thresh, contours)

    return result_image


# archivos de video de entrada y salida
filename_in = 'highway_600.mp4'
filename_out = 'highway_frame_difference_full.mp4'

# procesar el video
video_processor(filename_in, filename_out, process_frame_difference_full,
                max_time=10, draw_mode=1)

# video resultante
media.show_video(media.read_video(filename_out), fps=30)

# función para procesar el video
filename_out = 'highway_frame_difference_full_2.mp4'
video_processor(filename_in, filename_out, process_frame_difference_full,
                max_time=10, draw_mode=0)

#video resultante
media.show_video(media.read_video(filename_out), fps=30)

This browser does not support the video tag.


This browser does not support the video tag.
